<a href="https://www.kaggle.com/code/bsrsrc/rating-product-sorting-reviews-in-amazon?scriptVersionId=181137235" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div class="alert alert-primary" style="margin-top: 20px">


<h1><center>Rating Product & Sorting Reviews</center></h1>

</div>

## Business Problem

One of the most important problems in e-commerce is the accurate calculation of post-purchase ratings for products. Solving this problem means providing more customer satisfaction for e-commerce websites, highlighting the product for sellers, and ensuring a seamless shopping experience for buyers.

Another problem is the accurate sorting of reviews given to products. Since misleading reviews can directly affect the sale of the product, they will cause both financial loss and customer loss. By solving these 2 fundamental problems, e-commerce websites and sellers will increase their sales, while customers will complete their purchasing journey smoothly.

---

## Dataset Story

* This dataset containing Amazon product data includes various metadata with product categories.
* It includes user ratings and reviews for the product in the electronics category, which received the most reviews.

| Variable        | Description                              |
|-----------------|------------------------------------------|
| reviewerID      | User ID                                   |
| asin            | Product ID                                |
| reviewerName    | User Name                                 |
| helpful         | Helpful rating degree                    |
| reviewText      | Review                                    |
| overall         | Product rating                            |
| summary         | Review summary                            |
| unixReviewTime  | Review time (UNIX format)                |
| reviewTime      | Review time (Raw)                        |
| day_diff        | Number of days since review               |
| helpful_yes     | Number of helpful votes for the review    |
| total_vote      | Total number of votes for the review      |


In [1]:
#libraries
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime as dt
import scipy.stats as st
pd.set_option("display.float_format", lambda x: "%.4f" % x)

In [2]:
#dataset
df_ = pd.read_csv("/kaggle/input/amazon-review/amazon_review.csv")
df = df_.copy()
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0000,best deal around,1373673600,2013-07-13,513,0,0


---
### TASK 1: Calculate Average Rating Based on Recent Reviews and Compare it with Existing Average Rating.
---

In [3]:
def check_df(df, head=5):
    print("#################### Shape ###################")
    print(df.shape)
    print("#################### Types ###################")
    print(df.dtypes)
    print("#################### Head ###################")
    print(df.head(head))
    print("#################### Tail ###################")
    print(df.tail(head))
    print("#################### NA ###################")
    print(df.isnull().sum())
    print("#################### Quantiles ###################")
    print(df.describe([0, 0.05, 0.50, 0.95, 1]).T)
check_df(df)


#################### Shape ###################
(4915, 12)
#################### Types ###################
reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
day_diff            int64
helpful_yes         int64
total_vote          int64
dtype: object
#################### Head ###################
       reviewerID        asin  reviewerName helpful  \
0  A3SBTW3WS4IQSN  B007WTAJTO           NaN  [0, 0]   
1  A18K1ODH1I2MVB  B007WTAJTO          0mie  [0, 0]   
2  A2FII3I2MBMUIA  B007WTAJTO           1K3  [0, 0]   
3   A3H99DFEG68SR  B007WTAJTO           1m2  [0, 0]   
4  A375ZM4U047O79  B007WTAJTO  2&amp;1/2Men  [0, 0]   

                                          reviewText  overall  \
0                                         No issues.   4.0000   
1  Purchased this for my device, it worked as adv...   5.0000 

In [4]:
df.groupby("reviewerID")["total_vote"].head()
df["overall"].mean()

0       0
1       0
2       0
3       0
4       0
       ..
4910    0
4911    0
4912    0
4913    0
4914    0
Name: total_vote, Length: 4915, dtype: int64

4.587589013224822

In [5]:
df["reviewTime"] = pd.to_datetime(df["reviewTime"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   reviewerID      4915 non-null   object        
 1   asin            4915 non-null   object        
 2   reviewerName    4914 non-null   object        
 3   helpful         4915 non-null   object        
 4   reviewText      4914 non-null   object        
 5   overall         4915 non-null   float64       
 6   summary         4915 non-null   object        
 7   unixReviewTime  4915 non-null   int64         
 8   reviewTime      4915 non-null   datetime64[ns]
 9   day_diff        4915 non-null   int64         
 10  helpful_yes     4915 non-null   int64         
 11  total_vote      4915 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 460.9+ KB


In [6]:
df["day_diff"].sort_values()
df.loc[df["day_diff"] <= 30, "overall"].mean()
df.loc[((df["day_diff"] > 30) & (df["day_diff"] <= 90)), "overall"].mean()
df.loc[((df["day_diff"] > 90) & (df["day_diff"] <= 180)), "overall"].mean()
df.loc[df["day_diff"] > 180, "overall"].mean()

1494       1
2603       1
3742       1
2793       1
2629       1
        ... 
2382    1062
502     1063
1761    1063
2881    1063
3974    1064
Name: day_diff, Length: 4915, dtype: int64

4.742424242424242

4.803149606299213

4.649484536082475

4.573373327180434

In [7]:
def time_based_rated_average(df, w1=28, w2=26, w3=24, w4=22):
    return df.loc[df["day_diff"] <= 30, "overall"].mean() * w1 / 100 +\
    df.loc[((df["day_diff"] > 30) & (df["day_diff"] <= 90)), "overall"].mean() * w2 / 100 +\
    df.loc[((df["day_diff"] > 90) & (df["day_diff"] <= 180)), "overall"].mean() * w3 / 100 +\
    df.loc[df["day_diff"] > 180, "overall"].mean() * w4 / 100

time_based_rated_average(df,w1=28, w2=26, w3=24, w4=22)
    

4.6987161061560725

In [8]:
df["helpful_no"] =df["total_vote"] - df["helpful_yes"]
df["helpful_no"].value_counts()
df.head()
df = df[["reviewerID", "overall", "reviewTime", "day_diff", "helpful_yes", "helpful_no", "total_vote"]]
df.head()

helpful_no
0      4674
1       175
2        43
3         7
27        2
4         2
6         2
73        1
8         1
10        1
68        1
110       1
183       1
77        1
126       1
14        1
9         1
Name: count, dtype: int64

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0000,Four Stars,1406073600,2014-07-23,138,0,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0000,nothing to really say....,1356220800,2012-12-23,715,0,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0000,best deal around,1373673600,2013-07-13,513,0,0,0


,reviewerID,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote
0,A3SBTW3WS4IQSN,4.0000,2014-07-23,138,0,0,0
1,A18K1ODH1I2MVB,5.0000,2013-10-25,409,0,0,0
2,A2FII3I2MBMUIA,4.0000,2012-12-23,715,0,0,0
3,A3H99DFEG68SR,5.0000,2013-11-21,382,0,0,0
4,A375ZM4U047O79,5.0000,2013-07-13,513,0,0,0


---
## Task 2: Determine the 20 reviews to be displayed on the Product Detail Page for the product.
---
### Calculate the scores using the up-down difference score method, add them to the dataset, and examine 20 observations.
---

In [9]:
def score_up_down_diff(up, down):
    return up-down
df["score_up_down_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values(by="score_up_down_diff", ascending=False).head(20)

,reviewerID,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_up_down_diff
2031,A12B7ZMXFI6IXY,5.0000,2013-01-05,702,1952,68,2020,1884
4212,AVBMZZAFEKO58,1.0000,2013-05-08,579,1568,126,1694,1442
3449,AOEAD7DPLZE53,5.0000,2012-09-26,803,1428,77,1505,1351
317,A1ZQAQFYSXL5MQ,1.0000,2012-02-09,1033,422,73,495,349
3981,A1K91XXQ6ZEBQR,5.0000,2012-10-22,777,112,27,139,85
4596,A1WTQUOQ4WG9AI,1.0000,2012-09-22,807,82,27,109,55
1835,A1J6VSUM80UAF8,5.0000,2014-02-28,283,60,8,68,52
4672,A2DKQQIZ793AV5,5.0000,2014-07-03,158,45,4,49,41
4306,AOHXKM5URSKAB,5.0000,2012-09-06,823,51,14,65,37
315,A2J26NNQX6WKAU,5.0000,2012-08-13,847,38,10,48,28


---
### Calculate the scores using the average rating score method, add them to the dataset, and examine 20 observations.
---

In [10]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values(by="score_average_rating", ascending=False).head(20)

,reviewerID,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_up_down_diff,score_average_rating
4277,A35KXSU6AD1481,5.0000,2012-12-19,719,1,0,1,1,1.0000
2881,A3VSG5X7GPNNW6,5.0000,2012-01-10,1063,1,0,1,1,1.0000
1073,A2ZXEKQ2OBZLEE,5.0000,2013-08-13,482,1,0,1,1,1.0000
445,AIWBDRNBODLEA,4.0000,2013-12-18,355,1,0,1,1,1.0000
3923,A2PH4RGYVR34L,5.0000,2013-12-30,343,1,0,1,1,1.0000
435,AUH8I22ITG020,5.0000,2012-07-24,867,1,0,1,1,1.0000
2901,A28TRYU3FJ039C,5.0000,2013-05-13,574,1,0,1,1,1.0000
2204,AANX2UN8NPE22,2.0000,2013-06-14,542,1,0,1,1,1.0000
2206,A3KO3964CNP0XN,5.0000,2013-10-31,403,1,0,1,1,1.0000
3408,A20WUUD9EDWY4N,5.0000,2013-07-25,501,1,0,1,1,1.0000


---
### Calculate the scores using the wilson lower bound score method, add them to the dataset, and examine 20 observations.
---

In [11]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson alt sınırını hesaplar.

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasındaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebiir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters:
    -----------
    up : int
        Başarılı gözlemlerin sayısı.
    down : int
        Toplam gözlem sayısı.
    confidence : float
        Güven seviyesi. Varsayılan değer 0.95.

    Returns
    -------
    float: Wilson alt sınırı.
    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"], confidence=0.95), axis=1)
df.sort_values(by="wilson_lower_bound", ascending=False).head(20)

,reviewerID,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_up_down_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,5.0000,2013-01-05,702,1952,68,2020,1884,0.9663,0.9575
3449,AOEAD7DPLZE53,5.0000,2012-09-26,803,1428,77,1505,1351,0.9488,0.9365
4212,AVBMZZAFEKO58,1.0000,2013-05-08,579,1568,126,1694,1442,0.9256,0.9121
317,A1ZQAQFYSXL5MQ,1.0000,2012-02-09,1033,422,73,495,349,0.8525,0.8186
4672,A2DKQQIZ793AV5,5.0000,2014-07-03,158,45,4,49,41,0.9184,0.8081
1835,A1J6VSUM80UAF8,5.0000,2014-02-28,283,60,8,68,52,0.8824,0.7847
3981,A1K91XXQ6ZEBQR,5.0000,2012-10-22,777,112,27,139,85,0.8058,0.7321
3807,AFGRMORWY2QNX,3.0000,2013-02-27,649,22,3,25,19,0.8800,0.7004
4306,AOHXKM5URSKAB,5.0000,2012-09-06,823,51,14,65,37,0.7846,0.6703
4596,A1WTQUOQ4WG9AI,1.0000,2012-09-22,807,82,27,109,55,0.7523,0.6636


In [12]:
df.sort_values(by="wilson_lower_bound", ascending=False).head(5)
df.sort_values(by="score_up_down_diff", ascending=False).head(5)
df.sort_values(by="score_average_rating", ascending=False).head(5)

,reviewerID,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_up_down_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,5.0000,2013-01-05,702,1952,68,2020,1884,0.9663,0.9575
3449,AOEAD7DPLZE53,5.0000,2012-09-26,803,1428,77,1505,1351,0.9488,0.9365
4212,AVBMZZAFEKO58,1.0000,2013-05-08,579,1568,126,1694,1442,0.9256,0.9121
317,A1ZQAQFYSXL5MQ,1.0000,2012-02-09,1033,422,73,495,349,0.8525,0.8186
4672,A2DKQQIZ793AV5,5.0000,2014-07-03,158,45,4,49,41,0.9184,0.8081


,reviewerID,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_up_down_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,5.0000,2013-01-05,702,1952,68,2020,1884,0.9663,0.9575
4212,AVBMZZAFEKO58,1.0000,2013-05-08,579,1568,126,1694,1442,0.9256,0.9121
3449,AOEAD7DPLZE53,5.0000,2012-09-26,803,1428,77,1505,1351,0.9488,0.9365
317,A1ZQAQFYSXL5MQ,1.0000,2012-02-09,1033,422,73,495,349,0.8525,0.8186
3981,A1K91XXQ6ZEBQR,5.0000,2012-10-22,777,112,27,139,85,0.8058,0.7321


,reviewerID,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_up_down_diff,score_average_rating,wilson_lower_bound
4277,A35KXSU6AD1481,5.0000,2012-12-19,719,1,0,1,1,1.0000,0.2065
2881,A3VSG5X7GPNNW6,5.0000,2012-01-10,1063,1,0,1,1,1.0000,0.2065
1073,A2ZXEKQ2OBZLEE,5.0000,2013-08-13,482,1,0,1,1,1.0000,0.2065
445,AIWBDRNBODLEA,4.0000,2013-12-18,355,1,0,1,1,1.0000,0.2065
3923,A2PH4RGYVR34L,5.0000,2013-12-30,343,1,0,1,1,1.0000,0.2065
